# H1C IDR3.2 View and Flag JD axis
View averaged Spectrograms and flag JDs which are affected by RFI or system failures. Flags are written to file.

In [1]:
import os
import h5py
import numpy as np
from itertools import product
from hera_cal.utils import get_sun_alt, LST2JD

import sys
sys.path.append("/users/pkeller/code/H1C_IDR3.2/")

from closurelib import cptools as cp
from closurelib import plot

from ipywidgets import interact

import matplotlib.pyplot as plt

## Load Data

In [2]:
# data directory
ddir = "/lustre/aoc/projects/hera/pkeller/data/H1C_IDR3.2/sample/"

# triad names
trnames = ["EQ14", "EQ28"]

# field names
fnames = ["A", "B", "C", "D", "E"]

data = dict()

for fname, trname in product(fnames, trnames):
    name = f"{trname}_F{fname}"
    path = os.path.join(ddir, f"{name}_B2.h5")
    data.update({name: dict()})
    
    with h5py.File(path, "r") as f:
        data[name].update({"JD": f["JD"][()]})
        data[name].update({"LST": f["LST"][()]})
        data[name].update({"triads": f["triads"][()]})
        data[name].update({"eicp": f["eicp trmed"][()]})
        data[name].update({"FRQ": f["FRQ"][()]})

## Make Closure Phases

In [3]:
cp_data = dict()

for fname in fnames:
    data1 = data[f"EQ14_F{fname}"]
    data2 = data[f"EQ28_F{fname}"]
    cp_avg = np.angle((data1["eicp"] + data2["eicp"]).mean(0))
    cp_data.update({fname: dict({"cp": cp_avg, "JD": data1["JD"], "LST": data1["LST"], "FRQ": data1["FRQ"]})})

## Flagging
Fuctions for flagging individual JDs or ranges of JDs and repeated LST integrations.

In [4]:
def flag_jd(data, idx=[], badjd=[], goodjd=["*"], set_nan=True):
    """ 
    Flag a set of JDs or JD indices
    """
    shape = (len(data["JD"]), len(data["LST"]))
    
    if "*" in goodjd:
        goodjd = data["JD"]
        
    flags = np.zeros(shape).astype(bool)
    flags[idx] = True
    flags[np.where(np.in1d(data["JD"], badjd))] = True
    flags[np.where(~np.in1d(data["JD"], goodjd))] = True
    
    if set_nan:
        data["cp"][flags] = np.nan
    
    return flags


def flag_jd_range(data, jd_range, set_nan=True):
    """ 
    Flag JD ranges
    """
    jd, lst = data["JD"], data["LST"]
    shape = (len(jd), len(lst))
    flags = np.zeros(shape).astype(bool)
    jds = np.array([LST2JD(lst * np.pi / 12, j) for j in jd])
    jd_range = np.atleast_2d(jd_range)

    for [jdmin, jdmax] in jd_range:
        flags[(jds > jdmin) & (jds < jdmax)] = True
    
    if set_nan:
        data["cp"][flags] = np.nan
    
    return flags  
    
    
def flag_lst(data, set_nan=True):
    """ 
    Flag repeated LST integrations and when sun is above horizon
    """
    jd, lst = data["JD"], data["LST"]
    shape = (len(jd), len(lst))
    
    # flag repeated integrations
    flags = np.zeros(shape).astype(bool)
    lst_flags = cp.flag_repeated_slices(data["cp"], axis=-2, raxis=-1)
    badlst = np.where(lst_flags)
    flags[badlst[0], badlst[1]] = True
    
    # flags when sun is above horizon
    jds = [LST2JD(lst * np.pi / 12, j) for j in jd]
    sun_alt = get_sun_alt(jds)
    sun_idx = np.where(sun_alt > 0)
    flags[sun_idx[0], sun_idx[1]] = True
    
    if set_nan:
        data["cp"][flags] = np.nan
    
    return flags

flags = dict()

goodjd = np.loadtxt("/users/pkeller/code/H1C_IDR3.2/data/JD_IDR3_2.dat")
jdranges = np.loadtxt("/users/pkeller/code/H1C_IDR3.2/data/JD_range_flagged.dat")

## Plot Closure Phase Spectrograms
Each Spectrogram is an average over triad classes (EQ14, EQ28), polaisations and triads, where for the latter the geometric median was used to avoid bad triads (see trmed.py). This data product allows to inspect different JDs for RFI and system failures.

In [5]:
def plot_spectrogram(data, j):
    """ 
    Plot a closure phase spectrogram.
    """
    fig, ax = plt.subplots(figsize=(10, 7))
    ax = plot.spectrogram(data["cp"][j], data["LST"], data["FRQ"], cmap="twilight", ax=ax)
    ax.set_title(f"JD {data['JD'][j]}")
    plt.show()

### Field A

In [6]:
plot_A = lambda j : plot_spectrogram(cp_data["A"], j)
interact(plot_A, j=(0, len(cp_data["A"]["JD"])-1))

interactive(children=(IntSlider(value=27, description='j', max=54), Output()), _dom_classes=('widget-interact'…

<function __main__.<lambda>(j)>

In [7]:
idx = np.where(cp_data["A"]["JD"] < 2458039)[0]
badjd = 2458000 + np.array([59, 58])

fl = flag_jd(cp_data["A"], idx, badjd, goodjd)
fl = flag_lst(cp_data["A"])
fl = fl | flag_jd_range(cp_data["A"], jdranges)
flags.update({"A": fl})

/lustre/aoc/projects/hera/pkeller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


### Field B

In [8]:
plot_B = lambda j : plot_spectrogram(cp_data["B"], j)
interact(plot_B, j=(0, len(cp_data["B"]["JD"])-1))

interactive(children=(IntSlider(value=70, description='j', max=140), Output()), _dom_classes=('widget-interact…

<function __main__.<lambda>(j)>

In [9]:
idx = []
badjd = []

fl = flag_jd(cp_data["B"], idx, badjd, goodjd)
fl = flag_lst(cp_data["B"])
fl = fl | flag_jd_range(cp_data["B"], jdranges)
flags.update({"B": fl})

/lustre/aoc/projects/hera/pkeller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


### Field C

In [10]:
plot_C = lambda j : plot_spectrogram(cp_data["C"], j)
interact(plot_C, j=(0, len(cp_data["C"]["JD"])-1))

interactive(children=(IntSlider(value=74, description='j', max=149), Output()), _dom_classes=('widget-interact…

<function __main__.<lambda>(j)>

In [11]:
idx = []
badjd = []

fl = flag_jd(cp_data["C"], idx, badjd, goodjd)
fl = flag_lst(cp_data["C"])
fl = fl | flag_jd_range(cp_data["C"], jdranges)
flags.update({"C": fl})

/lustre/aoc/projects/hera/pkeller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


### Field D

In [12]:
plot_D = lambda j : plot_spectrogram(cp_data["D"], j)
interact(plot_D, j=(0, len(cp_data["D"]["JD"])-1))

interactive(children=(IntSlider(value=83, description='j', max=166), Output()), _dom_classes=('widget-interact…

<function __main__.<lambda>(j)>

In [13]:
idx = []
badjd = []

fl = flag_jd(cp_data["D"], idx, badjd, goodjd)
fl = flag_lst(cp_data["D"])
fl = fl | flag_jd_range(cp_data["D"], jdranges)
flags.update({"D": fl})

/lustre/aoc/projects/hera/pkeller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


### Field E

In [14]:
plot_E = lambda j : plot_spectrogram(cp_data["E"], j)
interact(plot_E, j=(0, len(cp_data["E"]["JD"])-1))

interactive(children=(IntSlider(value=77, description='j', max=154), Output()), _dom_classes=('widget-interact…

<function __main__.<lambda>(j)>

In [15]:
idx = []
badjd = []

fl = flag_jd(cp_data["E"], idx, badjd, goodjd)
fl = flag_lst(cp_data["E"])
fl = fl | flag_jd_range(cp_data["E"], jdranges)
flags.update({"E": fl})

/lustre/aoc/projects/hera/pkeller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


## Write Flags to Files

In [16]:
for fname, trname in product(fnames, trnames):
    name = f"{trname}_F{fname}"
    path = os.path.join(ddir, f"{name}_B2.h5")
    
    f = h5py.File(path, "a")
    if "JD-LST flags" in f.keys():
        del f["JD-LST flags"]
    f.create_dataset("JD-LST flags", data=flags[fname].astype(bool))
    f.close()

BlockingIOError: [Errno 11] Unable to open file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')